# Twitter RecSys Challenge 2020

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from twitter_preproc import twitter_preproc

#spark = SparkSession.builder.appName("ChiSquareSpark").getOrCreate()
spark = SparkSession \
    .builder \
    .appName("Pipeline") \
    .getOrCreate()
sc = spark.sparkContext

## Preproc Data Pipeline

In [132]:
%%file twitter_preproc.py
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import * 
from pyspark.ml.feature import RegexTokenizer, OneHotEncoderEstimator, StringIndexer

class twitter_preproc:
    
    def __init__(self, spark:SparkSession, sc:SparkContext, inputFile:str, seed:int=123, MF:bool=False):
        self.sc = sc
        #inputRDD = sc.textFile(inputFile)
        #self.inputData = spark.read.option("sep", "\x01").csv(inputFile)
        SCHEMA = StructType([
                StructField("text_tokens", StringType()),
                StructField("hashtags", StringType()),
                StructField("tweet_id", StringType()),
                StructField("present_media", StringType()),
                StructField("present_links", StringType()),
                StructField("present_domains", StringType()),
                StructField("tweet_type", StringType()),
                StructField("language", StringType()),
                StructField("tweet_timestamp", LongType()),
                StructField("engaged_with_user_id", StringType()),
                StructField("engaged_with_user_follower_count", LongType()),
                StructField("engaged_with_user_following_count", LongType()),
                StructField("engaged_with_user_is_verified", BooleanType()),
                StructField("engaged_with_user_account_creation", LongType()),
                StructField("engaging_user_id", StringType()),
                StructField("engaging_user_follower_count", LongType()),
                StructField("engaging_user_following_count", LongType()),
                StructField("engaging_user_is_verified", BooleanType()),
                StructField("engaging_user_account_creation", LongType()),
                StructField("engaged_follows_engaging", BooleanType()),
                StructField("reply_timestamp", LongType()),
                StructField("retweet_timestamp", LongType()),
                StructField("retweet_with_comment_timestamp", LongType()),
                StructField("like_timestamp", LongType())       
            ])
        self.inputData = spark.read.csv(path=inputFile, sep="\x01", header=False, schema=SCHEMA)
        if MF:
            self._preprocessMF()
        else:
            self._preprocess(seed)
        #self.inputData = spark.createDataFrame(inputRDD, sep="\x01", schema=SCHEMA)    
    
    def getDF(self):
        return self.outputDF
    
    def _preprocessMF(self):
        outputDF = self.inputData
        
        self.outputDF = outputDF.select(["tweet_id","engaging_user_id","engaged_with_user_id",
                                    "retweet_timestamp","reply_timestamp",
                                    "retweet_with_comment_timestamp","like_timestamp"])
    
    
    
    def _preprocess(self, seed):
        
        outputDF = self.inputData
        
        # Drop unnecessary cols
        ### drop ids for classification
        outputDF = outputDF.drop("tweet_id").drop("engaged_user_id").drop("engaged_with_user_id")\
                    .drop("present_links").drop("present_domains")
        
        # Split the text tokens to valid format
        regexTokenizer = RegexTokenizer(inputCol="text_tokens",outputCol="vector", pattern="\t")
        outputDF = regexTokenizer.transform(outputDF)
        outputDF = outputDF.drop("text_tokens").withColumnRenamed("vector", "text_tokens")
        
        regexTokenizer = RegexTokenizer(inputCol="present_media", outputCol="media_list")
        outputDF = regexTokenizer.transform(outputDF.fillna("none", subset=["present_media"]))
        outputDF = outputDF.drop("present_media").withColumnRenamed("media_list", "present_media")
        outputDF = outputDF.withColumn("present_media2", outputDF["present_media"].cast(StringType()))
        outputDF = outputDF.drop("present_media").withColumnRenamed("present_media2", "present_media")

        # OneHotEncode tweet_type
        ## TODO: user_id, engaged_user_id, ...
        indexer = StringIndexer(inputCol="tweet_type", outputCol="tweet_type_id")
        outputDF = indexer.fit(outputDF).transform(outputDF)
        indexer = StringIndexer(inputCol="present_media", outputCol="present_media_id")
        outputDF = indexer.fit(outputDF).transform(outputDF)
        indexer = StringIndexer(inputCol="language", outputCol="language_id")
        outputDF = indexer.fit(outputDF).transform(outputDF)
        
        # onehot
        encoder = OneHotEncoderEstimator(inputCols=["tweet_type_id", "present_media_id", "language_id"],
                                         outputCols=["tweet_type_onehot", "present_media_onehot", "language_onehot"])
        model = encoder.fit(outputDF)
        outputDF = model.transform(outputDF)
        
        # for explainability safe this
        self.explainOneHotDF = outputDF.select("tweet_type", "tweet_type_id", "tweet_type_onehot",
                                              "present_media", "present_media_id", "present_media_onehot",
                                               "language", "language_id", "language_onehot"
                                              )
        # make label columns binary
        outputDF = outputDF.withColumn("like", when(outputDF["like_timestamp"].isNull(), 0).otherwise(1))
        outputDF = outputDF.withColumn("retweet", when(outputDF["retweet_timestamp"].isNull(), 0).otherwise(1))
        outputDF = outputDF.withColumn("reply", when(outputDF["reply_timestamp"].isNull(), 0).otherwise(1))
        outputDF = outputDF.withColumn("retweet_comment", when(outputDF["retweet_with_comment_timestamp"].isNull(), 0).otherwise(1))
        
        # drop intermediate columns
        outputDF = outputDF.drop(*["like_timestamp","retweet_timestamp","reply_timestamp",
                                  "retweet_with_comment_timestamp","tweet_type","tweet_type_id",
                                 "language","language_id","present_media","present_media_id"])
        
        # TODO: Train/Test split and Scaling
        
        # might not need
        # transform boolean to 0-1 column... first one has to change the type in the schema though 
        #data = data.select("engaging_user_is_verified", "engaged_with_user_is_verified", "engaged_follows_engaging")\
        #    .replace(["false","true"], ["0","1"]).show()
        
        
        self.outputDF = outputDF
        
    '''
        returns small dataframe that explains the values of the oneHotEncoder step, this might be needed
        for mapping the encodings back to the original values
    '''    
    def explainOneHot(self):
        return self.explainOneHotDF


Overwriting twitter_preproc.py


## Interactive code...

In [133]:
#train = "///user/e11920598/traintweet_1000.tsv"
train = "///tmp/traintweet_1000.tsv"
#train = "///user/pknees/RSC20/training.tsv"

In [134]:
from pyspark.sql.types import *

column_names = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified",\
               "engaging_user_account_creation", "engaged_follows_engaging", "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]

SCHEMA = StructType([
                StructField("text_tokens", StringType()),
                StructField("hashtags", StringType()),
                StructField("tweet_id", StringType()),
                StructField("present_media", StringType()),
                StructField("present_links", StringType()),
                StructField("present_domains", StringType()),
                StructField("tweet_type", StringType()),
                StructField("language", StringType()),
                StructField("tweet_timestamp", LongType()),
                StructField("engaged_with_user_id", StringType()),
                StructField("engaged_with_user_follower_count", LongType()),
                StructField("engaged_with_user_following_count", LongType()),
                StructField("engaged_with_user_is_verified", BooleanType()),
                StructField("engaged_with_user_account_creation", LongType()),
                StructField("engaging_user_id", StringType()),
                StructField("engaging_user_follower_count", LongType()),
                StructField("engaging_user_following_count", LongType()),
                StructField("engaging_user_is_verified", BooleanType()),
                StructField("engaging_user_account_creation", LongType()),
                StructField("engaged_follows_engaging", BooleanType()),
                StructField("reply_timestamp", LongType()),
                StructField("retweet_timestamp", LongType()),
                StructField("retweet_with_comment_timestamp", LongType()),
                StructField("like_timestamp", LongType())       
                                ])

len(column_names)

24

In [135]:
import twitter_preproc
import importlib
importlib.reload(twitter_preproc)
from twitter_preproc import *

In [136]:
preproc = twitter_preproc(spark, sc, train, column_names, SCHEMA)
print(preproc.getDF().show(5))
#print(preproc.getDF().show(5))
import pandas as pd
pd.DataFrame(preproc.getDF().take(1000), columns=preproc.getDF().columns)

+--------------------+--------------------+--------------------+-----------------+---------------+------------------------------+--------------+
|            tweet_id|    engaging_user_id|engaged_with_user_id|retweet_timestamp|reply_timestamp|retweet_with_comment_timestamp|like_timestamp|
+--------------------+--------------------+--------------------+-----------------+---------------+------------------------------+--------------+
|E7D6C5094767223F6...|00000776B07587ECA...|D557B03872EF8986F...|             null|           null|                          null|          null|
|129F4A868712BA2B9...|00000B85AAF7DE172...|424822AC982CE0E89...|       1581497559|           null|                          null|    1581497622|
|04C6C2175852CDBBC...|00000E0C9B364891C...|1EC14E26417AA9260...|             null|           null|                          null|    1581060554|
|168157826315514C1...|00000F04EEDBCF3E1...|9B9595B6FEB8948BD...|             null|           null|                          null| 

,tweet_id,engaging_user_id,engaged_with_user_id,retweet_timestamp,reply_timestamp,retweet_with_comment_timestamp,like_timestamp
0,E7D6C5094767223F6F8789A87A1937AB,00000776B07587ECA9717BFC301F2D6E,D557B03872EF8986F7F4426AE094B2FE,NaN,NaN,NaN,NaN
1,129F4A868712BA2B98D31AF98C3066E4,00000B85AAF7DE172876FD96718C4469,424822AC982CE0E8965506C63B44EC12,1.581498e+09,NaN,NaN,1.581498e+09
2,04C6C2175852CDBBC23B2446C7E7C22D,00000E0C9B364891CDE89ECFC54771DE,1EC14E26417AA926095530AC591BA9CE,NaN,NaN,NaN,1.581061e+09
3,168157826315514C120494D4DF8E6216,00000F04EEDBCF3E1FB9A1948BF353B6,9B9595B6FEB8948BDDF0D222F27E0118,NaN,NaN,NaN,1.581329e+09
4,B3E3673782A69D9D8A45D3B222F0B073,000010088197DA00D659853E06935B3E,525DC99B7CB8F1AC4AD3E66C53FA38E0,NaN,NaN,NaN,1.580958e+09
5,AB21A06B694D637075F1EA4F89A05197,000012A6D58B300B1B4098C86223F76E,7E1E2FAD93219D0247BDBE451AB343E9,1.581347e+09,NaN,NaN,1.581347e+09
6,166C053A658691172A7A3CB20D8FB614,000012D4971A83624EF9C6711AE5167D,A0FD6DF4B4FBF62949708CDB97CC8124,NaN,1.581009e+09,NaN,NaN
7,C6016D70FDDAF88BB64B00600B48F788,000013E6563760E3916215D42BE0D406,629A622B84E4C67FAB56DCF0DBD785AA,NaN,NaN,NaN,NaN
8,293740496A195D5B20DBE00C3AEFFF17,00001607209C5774DF9207A2AC0EED5F,AD86A376FA5F26E67263D5FCA8A5BD59,NaN,NaN,NaN,1.581009e+09
9,3D89E8BE2E330DA8DD754D58EA07E824,0000170273D2530A0DF580401CC32AE0,2F236A7D11ECDAF1FC472E9ACC32AE6B,NaN,NaN,NaN,NaN


### Explanation

In [92]:
from pyspark.sql.functions import * 

data = preproc.getDF()
data.select("like_timestamp").show()
foo = data.withColumn("like", when(data["like_timestamp"].isNull(), 0).otherwise(1))
foo.select("like_timestamp", "like").show()
#data = data.drop("text_tokens").withColumnRenamed("vector", "text_tokens")
print("### Tweet-Type OneHotEncodings:")
explainonehot = preproc.explainOneHot()
explainonehot.show()
#data.show()
#data.groupBy("engaging_user_is_verified").count().show()
#data = data.select("engaging_user_is_verified", "engaged_with_user_is_verified", "engaged_follows_engaging")\
#    .replace(["false","true"], ["0","1"])..show()


+--------------+
|like_timestamp|
+--------------+
|          null|
|    1581497622|
|    1581060554|
|    1581328518|
|    1580957807|
|    1581346588|
|          null|
|          null|
|    1581009248|
|          null|
|    1581189873|
|          null|
|    1581045318|
|    1581375276|
|    1581063697|
|          null|
|    1581017998|
|          null|
|          null|
|    1581260483|
+--------------+
only showing top 20 rows

+--------------+----+
|like_timestamp|like|
+--------------+----+
|          null|   0|
|    1581497622|   1|
|    1581060554|   1|
|    1581328518|   1|
|    1580957807|   1|
|    1581346588|   1|
|          null|   0|
|          null|   0|
|    1581009248|   1|
|          null|   0|
|    1581189873|   1|
|          null|   0|
|    1581045318|   1|
|    1581375276|   1|
|    1581063697|   1|
|          null|   0|
|    1581017998|   1|
|          null|   0|
|          null|   0|
|    1581260483|   1|
+--------------+----+
only showing top 20 rows

### Tweet-Ty

In [73]:
data.select("present_media").show(5)
data.withColumn("present_media2", data["present_media"].cast(StringType()))
#data.select("present_media").rdd.map(lambda x: str(x[0])).toDF(schema= StructType([
#                StructField("present_media", StringType())])).show(5)


+-------------+
|present_media|
+-------------+
|       [none]|
|       [none]|
|       [none]|
|       [none]|
|      [photo]|
+-------------+
only showing top 5 rows



DataFrame[hashtags: string, tweet_type: string, language: string, tweet_timestamp: bigint, engaged_with_user_follower_count: bigint, engaged_with_user_following_count: bigint, engaged_with_user_is_verified: boolean, engaged_with_user_account_creation: bigint, engaging_user_id: string, engaging_user_follower_count: bigint, engaging_user_following_count: bigint, engaging_user_is_verified: boolean, engaging_user_account_creation: bigint, engaged_follows_engaging: boolean, reply_timestamp: bigint, retweet_timestamp: bigint, retweet_with_comment_timestamp: bigint, like_timestamp: bigint, text_tokens: array<string>, present_media: string, tweet_type_id: double, present_media_id: double, tweet_type_onehot: vector, present_media_onehot: vector, present_media2: string]

In [21]:
foo.foreach(print)
str(foo)

'PythonRDD[63] at RDD at PythonRDD.scala:53'

In [18]:
from pyspark.ml.feature import OneHotEncoderEstimator

df = spark.createDataFrame([
    (0.0, 1.0),
    (1.0, 0.0),
    (2.0, 1.0),
    (0.0, 2.0),
    (0.0, 1.0),
    (2.0, 0.0)
], ["categoryIndex1", "categoryIndex2"])

encoder = OneHotEncoderEstimator(inputCols=["categoryIndex1", "categoryIndex2"],
                                 outputCols=["categoryVec1", "categoryVec2"])
model = encoder.fit(df)
encoded = model.transform(df)
encoded.show()

+--------------+--------------+-------------+-------------+
|categoryIndex1|categoryIndex2| categoryVec1| categoryVec2|
+--------------+--------------+-------------+-------------+
|           0.0|           1.0|(2,[0],[1.0])|(2,[1],[1.0])|
|           1.0|           0.0|(2,[1],[1.0])|(2,[0],[1.0])|
|           2.0|           1.0|    (2,[],[])|(2,[1],[1.0])|
|           0.0|           2.0|(2,[0],[1.0])|    (2,[],[])|
|           0.0|           1.0|(2,[0],[1.0])|(2,[1],[1.0])|
|           2.0|           0.0|    (2,[],[])|(2,[0],[1.0])|
+--------------+--------------+-------------+-------------+



In [ ]:
from pyspark.ml.feature import RegexTokenizer

data = preproc.getDF()

regexTokenizer = RegexTokenizer(inputCol="text_tokens",outputCol="vector", pattern="\t")
tokenized = regexTokenizer.transform(data)

tokenized.select("vector").show()

In [ ]:
tokenized

In [7]:
%%file demo.py


from twitter_preproc import twitter_preproc
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

#spark = SparkSession.builder.appName("ChiSquareSpark").getOrCreate()
spark = SparkSession \
    .builder \
    .appName("Pipeline") \
    .getOrCreate()
sc = spark.sparkContext


# sample file with 1000 tweets for checking the pipeline
train = "///user/e11920598/traintweet_1000.tsv"

preproc = twitter_preproc(spark, sc, train)
print(preproc.getDF().show(5))


sc.stop()


Overwriting demo.py


In [8]:
### rather use it on the command line than here
#! spark-submit --num-executors=4 --total-executor-cores 16 --executor-memory=8G demo.py 

20/06/06 16:26:47 INFO spark.SparkContext: Running Spark version 2.4.0-cdh6.3.2
20/06/06 16:26:47 INFO logging.DriverLogger: Added a local log appender at: /tmp/spark-95acd95f-2bb4-4950-bc77-662ca74baeab/__driver_logs__/driver.log
20/06/06 16:26:47 INFO spark.SparkContext: Submitted application: Pipeline
20/06/06 16:26:47 INFO spark.SecurityManager: Changing view acls to: e11920598
20/06/06 16:26:47 INFO spark.SecurityManager: Changing modify acls to: e11920598
20/06/06 16:26:47 INFO spark.SecurityManager: Changing view acls groups to: 
20/06/06 16:26:47 INFO spark.SecurityManager: Changing modify acls groups to: 
20/06/06 16:26:47 INFO spark.SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(e11920598); groups with view permissions: Set(); users  with modify permissions: Set(e11920598); groups with modify permissions: Set()
20/06/06 16:26:47 INFO util.Utils: Successfully started service 'sparkDriver' on port 40223.
20/06/06 1